<a href="https://colab.research.google.com/github/jainam2718/120_JainamShah/blob/master/Lab05_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Linear Regression Model using PyTorch built-ins**

Let's re-implement the same model using some built-in functions and classes from PyTorch.

And now using two different targets: Apples and Oranges


In [1]:
# Import Numpy & PyTorch
import numpy as np
import torch
import torch.nn as nn

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print("Inputs : \n",inputs)
print("Targets : \n",targets)

Inputs : 
 tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
Targets : 
 tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])



**Dataset and DataLoader**

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples. We'll also create a DataLoader, to split the data into batches while training. It also provides other utilities like shuffling and sampling.


In [2]:
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [4]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [8]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.]])
tensor([[ 81., 101.],
        [119., 133.],
        [103., 119.],
        [ 56.,  70.],
        [119., 133.]])



**nn.Linear**

Instead of initializing the weights & biases manually, we can define the model using nn.Linear.


In [9]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1215, -0.5250, -0.4267],
        [-0.2251, -0.1994, -0.4310]], requires_grad=True)
Parameter containing:
tensor([ 0.5511, -0.3549], requires_grad=True)


In [10]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.1215, -0.5250, -0.4267],
         [-0.2251, -0.1994, -0.4310]], requires_grad=True),
 Parameter containing:
 tensor([ 0.5511, -0.3549], requires_grad=True)]

In [11]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -61.8431,  -48.6829],
        [ -84.0164,  -65.9738],
        [-105.1218,  -71.6614],
        [ -50.2052,  -47.8390],
        [ -88.1042,  -65.2026],
        [ -61.8431,  -48.6829],
        [ -84.0164,  -65.9738],
        [-105.1218,  -71.6614],
        [ -50.2052,  -47.8390],
        [ -88.1042,  -65.2026],
        [ -61.8431,  -48.6829],
        [ -84.0164,  -65.9738],
        [-105.1218,  -71.6614],
        [ -50.2052,  -47.8390],
        [ -88.1042,  -65.2026]], grad_fn=<AddmmBackward>)


**Loss Function**

Instead of defining a loss function manually, we can use the built-in loss function mse_loss.


In [12]:
# Import nn.functional
import torch.nn.functional as F
# Define loss function
loss_fn = F.mse_loss
loss = loss_fn(model(inputs) , targets)
print(loss)

tensor(25806.2910, grad_fn=<MseLossBackward>)



**Optimizer**

Instead of manually manipulating the weights & biases using gradients, we can use the optimizer optim.SGD.


In [17]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)


**Train the model**

We are ready to train the model now. We can define a utility function fit which trains the model for a given number of epochs.


In [16]:
# Define a utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
   for epoch in range(num_epochs):
       for xb,yb in train_dl:
            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred,yb)
            # Perform gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
   print('Training loss: ', loss_fn(model(inputs), targets))

# Train the model for 100 epochs
fit(100 , model , loss_fn, opt)

Training loss:  tensor(17.3654, grad_fn=<MseLossBackward>)


In [18]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.7117,  70.7372],
        [ 80.0579,  97.3433],
        [122.6267, 139.4886],
        [ 23.5267,  39.7541],
        [ 96.8698, 111.7081],
        [ 57.7117,  70.7372],
        [ 80.0579,  97.3433],
        [122.6267, 139.4886],
        [ 23.5267,  39.7541],
        [ 96.8698, 111.7081],
        [ 57.7117,  70.7372],
        [ 80.0579,  97.3433],
        [122.6267, 139.4886],
        [ 23.5267,  39.7541],
        [ 96.8698, 111.7081]], grad_fn=<AddmmBackward>)

In [19]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])